<a href="https://colab.research.google.com/github/Sameer-30/Damped-1D-Wave-Equation-Stress-Wave-Propagation-in-Structural-Members/blob/main/Damped_1D_Wave_Equation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
L = 1.0               # Length of the rod
nx = 200              # Number of spatial points
dx = L / (nx - 1)     # Spatial step size
x = np.linspace(0, L, nx)

c = 1.0               # Wave speed
zeta = 0.02           # Damping ratio
omega = np.pi * c / L # Natural frequency (first mode)

dt = 0.5 * dx / c     # Time step (CFL condition)
nt = 500              # Number of time steps

# Initial displacement: a Gaussian pulse
sigma = 0.05
u0 = np.exp(-((x - L/2)**2) / (2 * sigma**2))

# Initialize solution arrays
u_prev = u0.copy()
u_curr = u0.copy()

# Compute u at t = dt using a Taylor expansion (initial velocity = 0)
u_xx = np.zeros_like(u0)
u_xx[1:-1] = (u0[2:] - 2*u0[1:-1] + u0[:-2]) / dx**2
u_curr[1:-1] = u0[1:-1] + 0.5 * (c * dt)**2 * u_xx[1:-1]
u_curr[0] = 0
u_curr[-1] = 0

# Choose time steps at which to take snapshots
snapshot_steps = [0, nt//4, nt//2, nt-1]
snapshots = {step: None for step in snapshot_steps}
snapshots[0] = u0.copy()

# Time-stepping loop
for n in range(1, nt):
    u_next = np.zeros_like(u0)
    alpha2 = (c * dt / dx)**2

    # Interior update (central differences + damping)
    for j in range(1, nx-1):
        u_next[j] = (2*u_curr[j] - u_prev[j]
                     + alpha2 * (u_curr[j+1] - 2*u_curr[j] + u_curr[j-1])
                     - 2*zeta*omega*dt*(u_curr[j] - u_prev[j]))
    # Enforce boundary conditions
    u_next[0] = 0
    u_next[-1] = 0

    # Shift for next iteration
    u_prev, u_curr = u_curr, u_next

    # Store snapshots
    if n in snapshot_steps:
        snapshots[n] = u_curr.copy()

# Plot snapshots with grid
plt.figure(figsize=(8, 4))  # Same size as I used earlier

for step, u_snap in snapshots.items():
    plt.plot(x, u_snap, label=f't = {step*dt:.2f} s')  # Uses default color cycle

plt.xlabel('Position $x$')
plt.ylabel('Displacement $u(x,t)$')
plt.title('Damped Wave Propagation')

plt.grid(True, linestyle=':', linewidth=0.8)  # Dotted grid, subtle but visible
plt.legend()
plt.tight_layout()
plt.savefig('wave_propagation_plot_with_grid.png', dpi=300)
plt.show()


In [ ]:
import matplotlib.animation as animation

# Prepare the animation figure
fig, ax = plt.subplots(figsize=(8, 4))
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
lines = []
texts = []

# Set axis
ax.set_xlim(0, L)
ax.set_ylim(-1.1*np.max(u0), 1.1*np.max(u0))
ax.set_xlabel('Position $x$')
ax.set_ylabel('Displacement $u(x,t)$')
ax.set_title('Simultaneous Wave Propagation at Key Times')
ax.grid(True, linestyle=':', linewidth=0.8)

# Recompute full simulation and store u(x,t) at every time step
u_prev = u0.copy()
u_curr = u0.copy()
history = [u0.copy()]
for n in range(1, nt):
    u_next = np.zeros_like(u0)
    alpha2 = (c * dt / dx)**2
    for j in range(1, nx - 1):
        u_next[j] = (2 * u_curr[j] - u_prev[j]
                     + alpha2 * (u_curr[j + 1] - 2 * u_curr[j] + u_curr[j - 1])
                     - 2 * zeta * omega * dt * (u_curr[j] - u_prev[j]))
    u_next[0] = 0
    u_next[-1] = 0
    u_prev, u_curr = u_curr, u_next
    history.append(u_curr.copy())

# Convert selected steps to frame indexes
selected_steps = [0, nt // 4, nt // 2, nt - 1]
selected_histories = [[history[i]] for i in selected_steps]  # start with first frame for each
selected_counters = [i for i in selected_steps]

# Initialize lines and labels for each wave
for i, step in enumerate(selected_steps):
    line, = ax.plot([], [], lw=2, color=colors[i])
    lines.append(line)
    txt = ax.text(0.01, 0.9 - i*0.07, '', transform=ax.transAxes, fontsize=9, color=colors[i])
    texts.append(txt)

# Animate forward all selected histories
def update(frame):
    for i in range(len(selected_steps)):
        idx = min(selected_counters[i] + frame, nt - 1)
        lines[i].set_data(x, history[idx])
        texts[i].set_text(f't = {idx*dt:.2f} s')
    return lines + texts

ani = animation.FuncAnimation(fig, update, frames=200, interval=50, blit=True)

# Display in Colab
from IPython.display import HTML
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Parameters
L = 1.0             # Length of the domain
T = 2.0             # Total time
c = 1.0             # Wave speed
nx = 200            # Number of spatial points
nt = 400            # Number of time steps
dx = L / (nx - 1)
dt = T / nt
x = np.linspace(0, L, nx)

# Damping and frequency
zeta = 0.05
omega = np.pi

# Initial Gaussian pulse
u0 = np.exp(-400 * (x - L/2)**2)
u_prev = u0.copy()
u_curr = u0.copy()
history = [u0.copy()]

# Time stepping (finite difference)
for n in range(1, nt):
    u_next = np.zeros_like(u0)
    alpha2 = (c * dt / dx)**2
    for j in range(1, nx - 1):
        u_next[j] = (2 * u_curr[j] - u_prev[j]
                     + alpha2 * (u_curr[j+1] - 2 * u_curr[j] + u_curr[j-1])
                     - 2 * zeta * omega * dt * (u_curr[j] - u_prev[j]))
    u_next[0] = 0
    u_next[-1] = 0
    u_prev, u_curr = u_curr, u_next
    history.append(u_curr.copy())

# Select wave snapshots to animate
selected_steps = [0, nt//4, nt//2, nt-1]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
trail_length = 20  # Number of fading trail frames

# Plot setup
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_xlim(0, L)
ax.set_ylim(-1.1*np.max(u0), 1.1*np.max(u0))
ax.set_xlabel(r'$x$ (Position)', fontsize=12)
ax.set_ylabel(r'$u(x,t)$ (Displacement)', fontsize=12)
ax.set_title('Damped Stress-Wave Propagation', fontsize=14, fontweight='bold')
ax.grid(True, linestyle=':', linewidth=0.8)

# Initialize lines, dots, and text labels
lines = [ax.plot([], [], lw=2, color=colors[i])[0] for i in range(len(selected_steps))]
dots = [ax.plot([], [], 'o', color=colors[i])[0] for i in range(len(selected_steps))]
texts = [ax.text(0.01, 0.9 - i*0.07, '', transform=ax.transAxes, fontsize=9, color=colors[i])
         for i in range(len(selected_steps))]

# Optional time progress bar
timebar, = ax.plot([], [], color='black', lw=3)

# Animation update function
def update(frame):
  for artist in ax.lines[len(selected_steps)+1:] + ax.collections:
        artist.remove()
  # clear fading trails (fill_between artifacts)

  for i, step in enumerate(selected_steps):
        idx = min(step + frame, nt - 1)
        y = history[idx]

        # Fading trails
        for k in range(trail_length):
            t_idx = idx - k
            if t_idx < 0: continue
            fade = max(0, 1 - k / trail_length)
            ax.plot(x, history[t_idx], color=colors[i], alpha=fade, lw=1.5)

        # Main line and crest
        lines[i].set_data(x, y)
        peak_idx = np.argmax(np.abs(y))
        dots[i].set_data([x[peak_idx]], [y[peak_idx]]) # Changed this line
        texts[i].set_text(f't = {idx*dt:.2f} s')

    # Time progress bar
  bar_x = [0, L * frame / 200]
  bar_y = [-1.1*np.max(u0)] * 2
  timebar.set_data(bar_x, bar_y)

  return lines + dots + texts + [timebar]

# Animate
ani = animation.FuncAnimation(fig, update, frames=200, interval=50, blit=True)

# For Jupyter or Colab
from IPython.display import HTML
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Domain of the rod
L = 10
x = np.linspace(0, L, 500)
alpha = 4  # controls the spread
u0 = np.exp(-alpha * (x - L/2)**2)  # Gaussian pulse centered at L/2

# Plot setup
fig, ax = plt.subplots(figsize=(10, 3))
ax.plot(x, u0, label='Gaussian Pulse', color='black', linewidth=2)

# Draw the rod line
ax.hlines(0, 0, L, colors='gray', linestyles='solid', linewidth=3)

# Dashed vertical lines at observation points
for xp in [3, 7]:
    ax.axvline(x=xp, color='gray', linestyle='dashed')
    ax.text(xp, 1.1, f'Point {1 if xp==3 else 2}', ha='center', fontsize=10)

# Annotations and labels
ax.text(0, 0.2, '0', ha='center', fontsize=12)
ax.text(L, 0.2, 'L', ha='center', fontsize=12)
ax.text(L/2, max(u0)+0.1, 'Gaussian Pulse', ha='center', fontsize=12)
ax.text(L/2, max(u0)+0.3, r'$t = 0$', ha='center', fontsize=12)

# Axis labels
ax.set_xlabel('Position along rod, $x$', fontsize=12)
ax.set_ylabel('Displacement, $u(x,0)$', fontsize=12)
ax.set_title('Stress-Wave Propagation in a Damped Rod (Initial Condition)', fontsize=14)

# Aesthetics
ax.set_ylim(-0.2, 1.4)
ax.grid(True, linestyle=':', alpha=0.6)
ax.set_xticks([0, 3, 5, 7, 10])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()
